## Q to GMX conversion  

This notebook provides an example of Qtools/Qpyl library usage in Python.

It loads Q parameters (.lib and .prm) and the structure file (.pdb) of a single residue (in this case phenol) and converts it to Gromacs format (.itp and .gro).  
  
*Notes*:  
1.  The parameters might not be exactly the same as in original OPLS due to rounding errors, especially in the A,B -> $\sigma$,$\epsilon$ conversion.    
2.  The output from this script has been validated by comparing the zeroth-step energies (via gmx dump, v5.0.2) of Tyr ($\Delta E = 0.01 \% $) and Trp ($\Delta E = 0.4 \% ~ $) residues, produced with the generated topology and the topology built with GMX opls parameters via pdb2gmx on the same structure.  
2.1. Bonds to -C were removed from the GMX library to prevent 'dangling bond error' in pdb2gmx.  
2.2. Two impropers are missing in TRP/GMX_opls (on CD2 and CE2). They were removed from Q for the test.  
2.3. Fixed a typo in Qoplsaa.lib v1.2 in the TRP improper section (HH2  CH2  CE2   CZ3, should be HH2  CH2  CZ2   CZ3).  
2.4. TRP proper dihedrals differ by about 3% in GMX vs Q. The "problematic" dihedrals that do not match are defined explicitly in aminoacids.rtp and come from Kamiski et al (JPCB, 2001). These parameters appear to match "opls2005" (ffld_server -version 14 output). When removed, the difference drops to 0.4 % (rounding errors).

#### Load the modules

In [1]:
from __future__ import print_function, division, absolute_import
import time
from Qpyl.core.qparameter import QPrm
from Qpyl.core.qlibrary import QLib
from Qpyl.core.qstructure import QStruct
from Qpyl.core.qtopology import QTopology
from Qpyl.common import init_logger
# load the logger
logger = init_logger('Qpyl')

#### Load  Q parameters

Set `ignore_errors = True` if you experience issues with bad parameters/non-integer charges/...

In [2]:
ignore_errors = False

qlib = QLib("oplsaa", ignore_errors=ignore_errors)
qprm = QPrm("oplsaa", ignore_errors=ignore_errors)
qstr = QStruct("fnl.pdb", "pdb", ignore_errors=ignore_errors)
qlib.read_lib("fnl.lib")
qprm.read_prm("fnl.prm")
qtop = QTopology(qlib, qprm, qstr)

if len(qtop.residues) != 1:
    raise Exception("Only single residue allowed")
resname = qtop.residues[0].name

#### Make the GRO

In [3]:
crds = []

for atom in qtop.atoms:
    x, y, z = [crd/10.0 for crd in atom.struct.coordinates]  # A to nm
    crds.append("{:>5d}{:<5s}{:>5s}{:>5d}{:>8.3f}{:>8.3f}{:>8.3f}{:>8.4f}{:>8.4f}{:>8.4f}"
                "".format(1, resname, atom.name, atom.index, x, y, z, 0, 0, 0))

gro = """\
{} from Q
{:>5d}
{}
0.0 0.0 0.0
""".format(resname, len(qtop.atoms), "\n".join(crds))

#### Make the ITP 

In [4]:
typs, atms, bnds, angs, dihs, imps, pairs = [], [], [], [], [], [], set([])

In [5]:
for aprm in sorted(set([atom.prm for atom in qtop.atoms]), key=lambda x: x.prm_id):
    if aprm.lj_B < 1e-7 and aprm.lj_A < 1e-7:        
        sig, eps = 0, 0
    elif aprm.lj_B < 1e-7:
        # when B is 0, we need to tell GMX this by setting the B to a random (1) value and 
        # then setting the calculated "fake" sigma to a negative value
        # GMX will recalculate c6 (B) and c12 (A) from the fake sigma/epsilons and set c6=B=0
        # https://github.com/gromacs/gromacs/blob/5fb87d63ce5df628bfca85f1cebdbc845ec89b40/src/gromacs/gmxpreprocess/convparm.cpp#L100
        new_B = 1.0
        sig, eps = -(aprm.lj_A/new_B)**(2/6) / 10,  (new_B**4) / 4 / (aprm.lj_A**2) * 4.184
    else:
        sig, eps = (aprm.lj_A/aprm.lj_B)**(2/6) / 10,  (aprm.lj_B**4) / 4 / (aprm.lj_A**2) * 4.184
    atype = "op_{}".format(aprm.atom_type)

    typs.append("   {:<10s} {:<10s} {:>10.6f}   0.000   A   {:>15e} {:>15e}"
                "".format(atype, atype, aprm.mass, sig, eps))

In [6]:
charge_groups = qtop.residues[0].lib.charge_groups
for atom in qtop.atoms:
    atype = "op_{}".format(atom.prm.atom_type)
    charge_group = [i+1 for i, ch_grp in enumerate(charge_groups) if atom.name in ch_grp][0]
    atms.append("{:>5d}   {:<10s} {:>5d} {:5s} {:5s} {:5d} {:10.6f} {:10.6f}"
                "".format(atom.index, atype, 1, resname, atom.name,
                          charge_group, atom.charge, atom.prm.mass))

In [7]:
charge_groups = qtop.residues

In [8]:
for bond in qtop.bonds:
    a1, a2 = [atom.index for atom in bond.atoms]
    bnds.append("{:>5d} {:>5d} {:>5d} {:>10.6f} {:>10.3f}"
                "".format(a1, a2, 1, bond.prm.r0/10.0, bond.prm.fc*4.184*100))

In [9]:
for angle in qtop.angles:
    a1, a2, a3 = [atom.index for atom in angle.atoms]
    angs.append("{:>5d} {:>5d} {:>5d} {:>5d} {:>10.3f} {:>10.3f}"
                "".format(a1, a2, a3, 1, angle.prm.theta0, angle.prm.fc*4.184))

In [10]:
# Use type 5 dihedral (Fourier, GMX Manual 4.2.13, table 5.5)
dih_type = 5

for torsion in qtop.torsions:
    opls_torsion = [0, 0, 0, 0]   # F1, F2, F3, F4
    for prm in torsion.prm.get_prms():
        fc, mult, phase, npaths = prm
        mult = abs(mult)
        if int(mult) != mult or npaths != 1.0 or \
              (mult%2 == 0 and phase != 180.0) or \
              int(mult) not in (1,2,3,4):
            raise Exception("Bad parameter: " + str(torsion.prm))
        opls_torsion[abs(int(mult))-1] = fc * 2 * 4.184  # Q to ffld to kJ/mol
    c1, c2, c3, c4 = opls_torsion

# Conversion to RB (type 3)
#    f1, f2, f3, f4 = opls_torsion
#    c0 = (f2 + (f1+f3)/2.0)
#    c1 = ((-f1 + 3*f3)/2.0)
#    c2 = (-f2 + 4*f4)
#    c3 = (-2*f3)
#    c4, c5 = 0, 0
    
    a1, a2, a3, a4 = [a.index for a in torsion.atoms]
    dihs.append("{:>5d} {:>5d} {:>5d} {:>5d} {:>5d} {:>10.6f} {:>10.6f} {:>10.6f} {:>10.6f}"
                "".format(a1, a2, a3, a4, dih_type, c1, c2, c3, c4))
    
    # find 1-4 pairs
    # check that atoms don't share bonds/angles (four/five member rings)
    # avoid duplicates (six member rings)
    if not (set(torsion.atoms[0].bonds) & set(torsion.atoms[3].bonds)) and \
       not (set(torsion.atoms[0].angles) & set(torsion.atoms[3].angles)):
        pairs.add(tuple(sorted((a1, a4))))
pairs = sorted(["{:>5d} {:>5d} {:>5d}".format(a1, a4, 1) for a1, a4 in pairs])

In [11]:
# Use type 4 periodic improper dihedral (GMX Manual 4.2.12, table 5.5)
imp_type = 4

for improper in qtop.impropers:
    a1, a2, a3, a4 = [a.index for a in improper.atoms]
    imps.append("{:>5d} {:>5d} {:>5d} {:>5d} {:>5d} {:>10.3f} {:>10.5f} {:>10.3f}"
                "".format(a1, a2, a3, a4, imp_type, improper.prm.phi0,
                          improper.prm.fc*4.184, improper.prm.multiplicity))

In [12]:
prms = {"atomtypes": typs,
        "atoms": atms,
        "bonds": bnds,
        "angles" : angs,
        "dihedrals": dihs,
        "impropers": imps,
        "pairs": pairs}
for k, v in prms.iteritems():
    prms[k] = "\n".join(v)

itp = """;
; OPLS/AA topology for '{resname}'
; Converted from Q with q2gmx.ipynb
; Date: {date}
;

[ atomtypes ]
; name mass charge ptype sigma(nm) epsilon (kJ/mol) 
{atomtypes}

[ moleculetype ]
; Name nrexcl
{resname} 3

[ atoms ]
; nr type  resnr residue  atom  cgnr charge  mass 
{atoms}

[ bonds ]
; ai    aj    type     r0 (nm)   fc (kJ/(mol nm2)) 
{bonds}

[ angles ]
;  ai    aj    ak type    theta0 (degr)   fc (kJ/(mol rad2) 
{angles}

[ dihedrals ]
; Type 5 Fourier
;  ai    aj    ak    al  type     coefficients
{dihedrals}

[ dihedrals ]
; Periodic improper dihedrals (type 4)
;  ai    aj    ak    al  type     phi0    fc (kJ/mol)   n 
{impropers}

[ pairs ]
;  ai    aj       f_qq    qi     qj   sigma (nm)  epsilon (kJ/mol) 
{pairs}
""".format(resname=resname, date=time.ctime(), **prms)

#### Write files

In [13]:
#open(resname+".gro", "w").write(gro)
#open(resname+".itp", "w").write(itp)

In [14]:
print(gro)

FNL from Q
   13
    1FNL     C1    1   0.141  -0.000   0.000  0.0000  0.0000  0.0000
    1FNL     C2    2   0.070   0.122   0.000  0.0000  0.0000  0.0000
    1FNL     C3    3  -0.070   0.122   0.000  0.0000  0.0000  0.0000
    1FNL     C4    4  -0.140  -0.000   0.000  0.0000  0.0000  0.0000
    1FNL     C5    5  -0.070  -0.122   0.000  0.0000  0.0000  0.0000
    1FNL     C6    6   0.070  -0.122   0.000  0.0000  0.0000  0.0000
    1FNL     O1    7   0.277  -0.000   0.000  0.0000  0.0000  0.0000
    1FNL     H1    8   0.125   0.217   0.000  0.0000  0.0000  0.0000
    1FNL     H2    9  -0.125   0.217   0.000  0.0000  0.0000  0.0000
    1FNL     H3   10  -0.251  -0.000   0.000  0.0000  0.0000  0.0000
    1FNL     H4   11  -0.125  -0.217   0.000  0.0000  0.0000  0.0000
    1FNL     H5   12   0.125  -0.217   0.000  0.0000  0.0000  0.0000
    1FNL     H6   13   0.303  -0.092   0.000  0.0000  0.0000  0.0000
0.0 0.0 0.0



In [15]:
print(itp)

;
; OPLS/AA topology for 'FNL'
; Converted from Q with q2gmx.ipynb
; Date: Mon Jul  3 18:18:35 2017
;

[ atomtypes ]
; name mass charge ptype sigma(nm) epsilon (kJ/mol) 
   op_fnl.C1  op_fnl.C1   12.011000   0.000   A      3.549999e-01    2.928806e-01
   op_fnl.C2  op_fnl.C2   12.011000   0.000   A      3.549999e-01    2.928806e-01
   op_fnl.C3  op_fnl.C3   12.011000   0.000   A      3.549999e-01    2.928806e-01
   op_fnl.C4  op_fnl.C4   12.011000   0.000   A      3.549999e-01    2.928806e-01
   op_fnl.C5  op_fnl.C5   12.011000   0.000   A      3.549999e-01    2.928806e-01
   op_fnl.C6  op_fnl.C6   12.011000   0.000   A      3.549999e-01    2.928806e-01
   op_fnl.H1  op_fnl.H1    1.007900   0.000   A      2.419998e-01    1.255208e-01
   op_fnl.H2  op_fnl.H2    1.007900   0.000   A      2.419998e-01    1.255208e-01
   op_fnl.H3  op_fnl.H3    1.007900   0.000   A      2.419998e-01    1.255208e-01
   op_fnl.H4  op_fnl.H4    1.007900   0.000   A      2.419998e-01    1.255208e-01
   op_fnl.